In [1]:
import os

# 환경변수에서 Stability.ai API 키 읽어옴.
stb_ai_api_key = os.getenv('stb_ai_api_key')

# API 키가 환경변수에 설정되어 있는지 확인.
if not stb_ai_api_key:
    raise ValueError("환경변수 'stb_ai_api_key'가 설정되지 않았습니다.")

In [8]:
# 해당 코드는 Stable Diffusion 전용 (Image to Image)
import requests
import json

url = "https://modelslab.com/api/v6/realtime/text2img"

payload = json.dumps({
    "key" : sd_api_key,
    "prompt": "ultra realistic close up portrait ((beautiful pale cyberpunk female with heavy black eyeliner))",
    "negative_prompt": "bad quality",
    "width": "512",
    "height": "512",
    "safety_checker": False,
    "seed": None,
    "samples":1,
    "base64":False,
    "webhook": None,
    "track_id": None
})

headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
response_data=json.loads(response.text)

print(json.dumps(response_data, indent=4))

{
    "message": "Unauthenticated."
}


In [9]:
import requests

response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/generate/ultra",
    headers={
        "authorization": stb_ai_api_key,
        "accept": "image/*"
    },
    files={"none": ''},
    data={
        "prompt": "Lighthouse on a cliff overlooking the ocean",
        "output_format": "webp",
    },
)

if response.status_code == 200:
    with open("./lighthouse.webp", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

In [5]:
import gradio as gr
with gr.Blocks(theme=gr.themes.Default()) as app:
    with gr.Tab("삽화 생성"):
        with gr.Row():
            #1
            gr.Markdown(
                value="""
                # <center>삽화 생성</center>
                <center>짧은 소설에 어울리는 이미지를 생성할 수 있습니다.</center>
                """)
        with gr.Row():
            with gr.Column():
                #2
                pos_prompt=gr.Textbox(
                    label="이미지 생성 프롬프트를 작성해 주세요."
                )
                with gr.Row():
                    #3
                    auto_prompt_generator=gr.Textbox(
                        label="자동 프롬프트 생성",
                    )
                    #4
                    prompt_generator_btn=gr.Button(
                        value="자동\n생성")
                with gr.Group():
                    with gr.Row():
                        #5
                        img_width=gr.Slider(
                            label="Width"
                        )
                        #6
                        img_height=gr.Slider(
                            label="Height"
                        )
        with gr.Column():
            #7
            output_status=gr.Textbox(
                label="이미지 상태 출력"
            )
            #8
            output_img=gr.Image(
                label="이미지가 생성됩니다."
            )
            #9
            generator_img=gr.Button(
                value="이미지 생성"
            )
            #10
            refresh_img_btn=gr.Button(
                value="이미지 새로고침"
            )

app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [35]:
import gradio as gr
with gr.Blocks(theme=gr.themes.Default()) as app:
    with gr.Tab("삽화 생성"):
        with gr.Row():
            #1
            gr.Markdown(
                value="""
                # <center>삽화 생성</center>
                <center>짧은 소설에 어울리는 이미지를 생성할 수 있습니다.</center>
                """)
        with gr.Row():
            # column1
            with gr.Column(scale=5):
                #2
                pos_prompt=gr.Textbox(
                    label="이미지 생성 프롬프트를 작성해 주세요.",
                    value="ultra realistic close up portrait ((beautiful pale cyberpunk female with heavy black eyeliner))",
                    lines=8,
                    interactive=True
                )
                with gr.Row():
                    #3
                    auto_prompt_generator=gr.Textbox(
                        label="자동 프롬프트 생성",
                        lines=6,
                        placeholder="이미지 생성을 위한 소설 내용을 작성해 주세요. 자동으로 프롬프트가 생성됩니다.",
                        scale=7
                    )
                    #4
                    prompt_generator_btn=gr.Button(
                        scale=1,
                        value="자동\n생성")
                with gr.Group():
                    with gr.Row():
                        #5
                        img_width=gr.Slider(
                            label="Width",
                            maximum=1024,
                            value=512,
                            interactive=True
                        )
                        #6
                        img_height=gr.Slider(
                            label="Height",
                            maximum=1024,
                            value=512,
                            interactive=True
                        )
        # column2
        with gr.Column():
            #7
            output_status=gr.Textbox(
                show_label=False,
                lines=1,
                placeholder="이미지 상태가 출력됩니다."
            )
            #8
            generator_img=gr.Image(
                value="https://pub-3626123a908346a7a8be8d9295f44e26.r2.dev/generations/f20412ca-cb22-4e77-9b75-26ddbad5a4e5-0.png",
                label="이미지가 생성됩니다."
            )
            #9
            generator_img_btn=gr.Button(
                value="이미지 생성"
            )
            #10
            refresh_img_btn=gr.Button(
                value="이미지 새로고침"
            )

app.launch()

Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


In [55]:
import os
import openai
import requests
import json
import gradio as gr

# 환경변수에서 Stability AI API 키 읽어옴.
stb_ai_api_key = os.getenv('stb_ai_api_key')

# API 키가 환경변수에 설정되어 있는지 확인.
if not stb_ai_api_key:
    raise ValueError("환경변수 'stb_ai_api_key'가 설정되지 않았습니다.")

# 환경변수에서 OpenAI API 키를 읽어옴.
openai_api_key = os.getenv('OPENAI_API_KEY')

openai.api_key = openai_api_key

# 자동 프롬프트 생성
def novel_keyword(novel_input):
    system_prompt = "당신은 주어지는 내용에 어울리는 이미지를 추천하는 역할을 합니다. 이미지를 묘사하는 키워드를 영어로, 콤마로 알려주세요."
    client = openai.OpenAI()
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": novel_input}
        ])
    return completion.choices[0].message.content

# Stability AI 이미지 생성 API 호출 함수
def st_call(prompt, width, height, retry_count=3):
    url = "https://api.stability.ai/v2beta/stable-image/generate/ultra"
    payload = json.dumps({
        "key": stb_ai_api_key,
        "text_prompts": [
            {
                "text": prompt,
                "weight": 1.0
            }
        ],
        "cfg_scale": 7.5,
        "clip_guidance_preset": "FAST_BLUE",
        "height": height,
        "width": width,
        "samples": 1,
        "steps": 50
    })
    headers = {
        'Content-Type': 'application/json',
    }
    response=requests.request("POST", url, headers=headers, data=payload)
    response_data=json.loads(response.text)
    if response_data["status"] == "success":
        image_link=response_data["output"][0]
        fetch_result="이미지 생성이 완료되었습니다."
    elif response_data["status"] == "processing":
        image_link=None
        fetch_result=response_data["fetch_result"]
    else:
        image_link=None
        fetch_result="실패입니다. 다시 실행해 주세요."
    return image_link, fetch_result

def st_recall(fetch_result):
    url=fetch_result
    payload=json.dumps({
        "key": stb_ai_api_key
    })
    headers = {
        'Content-Type': 'application/json',
    }
    response=requests.request("POST", url, headers=headers, data=payload)
    response_data=json.loads(response.text)

    image_link=response_data["output"]
    if not image_link:
        image_link=None
    else:
        image_link=image_link[0]
    return image_link

# Gradio 인터페이스 정의
with gr.Blocks(theme=gr.themes.Default()) as app:
    with gr.Tab("삽화 생성"):
        with gr.Row():
            gr.Markdown(
                value="""
                # <center>삽화 생성</center>
                <center>짧은 소설에 어울리는 이미지를 생성할 수 있습니다.</center>
                """
            )
        with gr.Row():
            with gr.Column(scale=5):
                pos_prompt = gr.Textbox(
                    label="이미지 생성 프롬프트를 작성해 주세요.",
                    value="",
                    lines=8,
                    interactive=True
                )
                with gr.Row():
                    auto_prompt_generator = gr.Textbox(
                        label="자동 프롬프트 생성",
                        lines=6,
                        placeholder="이미지 생성을 위한 소설 내용을 작성해 주세요. 자동으로 프롬프트가 생성됩니다.",
                        scale=7
                    )
                    prompt_generator_btn = gr.Button(
                        scale=1,
                        value="자동\n생성"
                    )
                    prompt_generator_btn.click(
                        lambda x: novel_keyword(x), 
                        inputs=[auto_prompt_generator], 
                        outputs=[pos_prompt]
                    )
                with gr.Group():
                    with gr.Row():
                        img_width = gr.Slider(
                            label="Width",
                            maximum=1024,
                            value=512,
                            interactive=True
                        )
                        img_height = gr.Slider(
                            label="Height",
                            maximum=1024,
                            value=512,
                            interactive=True
                        )
        with gr.Column():
            output_status = gr.Textbox(
                show_label=False,
                lines=1,
                placeholder="이미지 상태가 출력됩니다."
            )
            generator_img = gr.Image(
                value="https://via.placeholder.com/512",
                label="이미지가 생성됩니다."
            )
            generator_img_btn = gr.Button(
                value="이미지 생성"
            )
            generator_img_btn.click(
                lambda p, w, h: st_call(p, w, h), 
                inputs=[pos_prompt, img_width, img_height], 
                outputs=[generator_img, output_status]
            )
app.launch()

Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\gradio\blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\anyio\_backends\_asyncio.py", line 2505, in run_sync_in_worker_thread
    return a

In [73]:
# 챗gpt
import os
import requests
import gradio as gr
import base64
import json

# 환경변수에서 Stability AI API 키 읽어옴.
stb_ai_api_key = os.getenv('stb_ai_api_key')

# Stability AI 이미지 생성 API 호출 함수
def st_call(prompt, retry_count=3):
    url = "[https://api.stability.ai/v2beta/stable-image/generate/sd3](https://api.stability.ai/v2beta/stable-image/generate/sd3)"
    headers = {
        'Authorization': f'Bearer {stb_ai_api_key}',
        'Content-Type': 'application/json'
    }
    payload = {
        "text_prompts": [{"text": prompt, "weight": 1}],
        "cfg_scale": 7.5,
        "height": 512,
        "width": 512,
        "samples": 1,
        "steps": 50
    }

    for attempt in range(retry_count):
        try:
            print(f"Attempt {attempt + 1}")
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()  # HTTP 오류에 대한 예외 처리

            # 이미지 데이터를 받아 base64로 인코딩된 데이터를 반환
            if response.status_code == 200:
                response_data = response.json()
                if 'artifacts' in response_data and len(response_data['artifacts']) > 0:
                    image_data = response_data['artifacts'][0]['base64']
                    return f"data:image/png;base64,{image_data}", "이미지 생성이 완료되었습니다."
                else:
                    return None, "이미지 생성 실패. 다시 시도해 주세요."
            else:
                return None, f"이미지 생성 실패: {response.content}"

        except requests.exceptions.RequestException as req_err:
            print(f"요청 오류 발생: {req_err}")
            return None, f"이미지 생성 실패. 요청 오류 발생: {req_err}"
        except Exception as err:
            print(f"예상치 못한 오류 발생: {err}")
            return None, f"이미지 생성 실패. 예상치 못한 오류 발생: {err}"

# Gradio 인터페이스 정의
def generate_image(prompt):
    return st_call(prompt)

with gr.Blocks(theme=gr.themes.Default()) as app:
    with gr.Tab("삽화 생성"):
        gr.Markdown(
            """
            # <center>삽화 생성</center>
            <center>소설 내용을 입력하면 어울리는 이미지를 생성합니다.</center>
            """
        )
        
        with gr.Row():
            with gr.Column(scale=5):
                pos_prompt = gr.Textbox(
                    label="이미지 생성 프롬프트를 작성해 주세요.",
                    lines=8,
                    interactive=True
                )
                
        with gr.Row():
            generate_btn = gr.Button(value="이미지 생성")
        
        with gr.Row():
            output_status = gr.Textbox(
                show_label=False,
                lines=1,
                placeholder="이미지 상태가 출력됩니다."
            )
            generator_img = gr.Image(
                value=None,
                label="생성된 이미지가 표시됩니다."
            )

        generate_btn.click(
            generate_image, 
            inputs=[pos_prompt], 
            outputs=[generator_img, output_status]
        )

app.launch()

Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


Attempt 1
요청 오류 발생: No connection adapters were found for '[https://api.stability.ai/v2beta/stable-image/generate/sd3](https://api.stability.ai/v2beta/stable-image/generate/sd3)'
